<a href="https://colab.research.google.com/github/andrefs/mapi-faml-proj/blob/main/code/faml.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Load stuff


## Libraries

In [ ]:
import csv
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error 
from keras.callbacks import ModelCheckpoint
from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten

from matplotlib import pyplot as plt
import seaborn as sb
import matplotlib.pyplot as plt
import pandas as pd
import warnings 
warnings.filterwarnings('ignore')
warnings.filterwarnings('ignore', category=DeprecationWarning)
from xgboost import XGBRegressor

## Embeddings

In [ ]:
# currently not being used
def normalize(word_vec):
    norm=np.linalg.norm(word_vec)
    if norm == 0: 
       return word_vec
    return word_vec/norm

embs = {}
url = 'https://raw.githubusercontent.com/andrefs/mapi-faml-proj/main/2_clean_datasets/embeddings.txt'
response = urllib.request.urlopen(url)
lines = [l.decode('utf-8') for l in response.readlines()]
reader = csv.reader(lines, delimiter=' ')
for line in reader:
    term = line[0].replace('http://dbpedia.org/resource/' , '')
    vector = [float(x) for x in line[1:]]
    embs[term] = vector


len(embs.keys())

## Relatedness pairs

In [ ]:
relness = {}
url = 'https://raw.githubusercontent.com/andrefs/mapi-faml-proj/main/2_clean_datasets/relatedness_train.tsv'
response = urllib.request.urlopen(url)
lc = 0
lines = [l.decode('utf-8') for l in response.readlines()]
reader = csv.reader(lines, delimiter='\t')
for line in reader:
    lc += 1
    t1 = line[0].replace('http://dbpedia.org/resource/' , '')
    t2 = line[1].replace('http://dbpedia.org/resource/' , '')
    rel = float(line[2])
    relness[t1] = relness.get(t1,{})
    relness[t1][t2] = [float(line[2]), line(3)]

lc